In [131]:
#TODO Functionality with meters
#TODO Issues with no 'On' Potential 

In [132]:
# Standard library imports for filesystem and execution time
import os
import shutil
import time

# Standard library import for networking
import socket

# Standard library imports for mathematical functions
import math
from math import atan, atan2, cos, radians, sin, sqrt, tan

# Third-party imports for data handling
import pandas as pd
import numpy as np
from natsort import natsorted

# Third-party imports for KML (Keyhole Markup Language) and Excel (`xlsx`) files handling
import simplekml
from openpyxl import load_workbook
from openpyxl.styles import Font, Alignment, Border, Side

# Third-party import for filesystem path names pattern matching
import glob

# Settings for pandas
pd.options.mode.chained_assignment = None

# Standard library import for datetime
import datetime

# Measuring execution time
start_time = time.time()

# Specify Paths

In [133]:
root_path = os.path.abspath('../')
data_folder_path = os.path.join(root_path, 'Data')
output_path = os.path.join(root_path, 'Output')
log_path = os.path.join(output_path, 'Log.txt')
duplicates_path = os.path.join(output_path, 'Duplicate GPS Pairs.CSV')

# Specify Variables

In [134]:
# Data Headers: 'Station, On Potential, Off Potential, Comments, Latitude, Longitude, ElevationPDOP'
# File Name: COMPANY LINE SNXXXX TO SNXXXXX

In [135]:
# SPECIFY ICONS
ICON_ON = 'https://img.icons8.com/ios-filled/50/22C3E6/filled-circle.png'
ICON_OFF = 'https://img.icons8.com/ios-filled/50/00FF00/filled-circle.png'
ICON_1200 = 'https://img.icons8.com/ios-filled/50/7950F2/filled-circle.png'
ICON_850 = 'https://img.icons8.com/ios-filled/50/F25081/filled-circle.png'
ICON_COMMENTS = 'https://img.icons8.com/ultraviolet/80/000000/comments.png'

# COMPANY
COMPANY_NAME = 'AMP'

# DIVISION BY 1000 (True or False)
DIVISION_1000 = False

# REVERSE DATA ROWS (True or False)
reverse = False

# SCALE FACTOR (Elevation of points, Default = 100)
SCALE_FACTOR = 100

# ICON SCALE (Default = 0.3)
ICON_SCALE = 0.3

# DATA VISIBILITY WHEN OPENING KMZ FILE (True or False)
DATA_VISIBILITY = False

# Vincenty Formula (0.5 mm Accuracy)

In [136]:
class VincentyInverse:
    def __init__(self, coord1, coord2, maxiter=200, tol=10 ** -12):
        # CONSTANTS
        a = 6378137.0  # radius at equator in meters (WGS-84)
        f = 1 / 298.257223563  # flattening of the ellipsoid (WGS-84)
        b = (1 - f) * a
        phi_1, lon_1, = coord1  # (lat=L_?,lon=phi_?)
        phi_2, lon_2, = coord2
        u_1 = atan((1 - f) * tan(radians(phi_1)))
        u_2 = atan((1 - f) * tan(radians(phi_2)))
        L = radians(lon_2 - lon_1)
        lamda = L  # set initial value of lambda to L
        sin_u1 = sin(u_1)
        cos_u1 = cos(u_1)
        sin_u2 = sin(u_2)
        cos_u2 = cos(u_2)

        # ITERATIONS
        self.iters = 0
        for i in range(0, maxiter):
            self.iters += 1
            cos_lambda = cos(lamda)
            sin_lambda = sin(lamda)
            sin_sigma = sqrt((cos_u2 * sin(lamda)) ** 2 +
                             (cos_u1 * sin_u2 - sin_u1 * cos_u2 * cos_lambda) ** 2)
            cos_sigma = sin_u1 * sin_u2 + cos_u1 * cos_u2 * cos_lambda
            sigma = atan2(sin_sigma, cos_sigma)
            sin_alpha = (cos_u1 * cos_u2 * sin_lambda) / sin_sigma
            cos_sq_alpha = 1 - sin_alpha ** 2
            cos2_sigma_m = cos_sigma - ((2 * sin_u1 * sin_u2) / cos_sq_alpha)
            C = (f / 16) * cos_sq_alpha * (4 + f * (4 - 3 * cos_sq_alpha))
            Lambda_prev = lamda
            lamda = L + (1 - C) * f * sin_alpha * (sigma + C * sin_sigma *
                                                   (cos2_sigma_m + C * cos_sigma *
                                                    (-1 + 2 * cos2_sigma_m ** 2)))

            # Successful convergence
            diff = abs(Lambda_prev - lamda)
            if diff <= tol:
                break

        u_sq = cos_sq_alpha * ((a ** 2 - b ** 2) / b ** 2)
        A = 1 + (u_sq / 16384) * (4096 + u_sq * (-768 + u_sq * (320 - 175 * u_sq)))
        B = (u_sq / 1024) * (256 + u_sq * (-128 + u_sq * (74 - 47 * u_sq)))
        delta_sig = B * sin_sigma * (cos2_sigma_m + 0.25 * B *
                                     (cos_sigma * (-1 + 2 * cos2_sigma_m ** 2) -
                                      (1 / 6) * B * cos2_sigma_m * (-3 + 4 * sin_sigma ** 2) *
                                      (-3 + 4 * cos2_sigma_m ** 2)))

        self.m = b * A * (sigma - delta_sig)  # output distance in meters
        self.miles = self.m * 0.000621371  # output distance in miles
        self.ft = self.miles * 5280  # output distance in feet

# Data Cleaning

In [137]:
# List of files contained in the original data folder
raw_data_list = os.listdir(os.path.abspath('../' + '/Original Data'))

In [138]:
# Replaces data folder with an empty one
if os.path.exists(data_folder_path):
    shutil.rmtree(data_folder_path)

os.makedirs(data_folder_path)

In [139]:
# Replaces output folder with an empty one
if os.path.exists(output_path):
    shutil.rmtree(output_path)

os.makedirs(output_path)

In [140]:
# Removes existing log file, if it exists
if os.path.exists(log_path):
    os.remove(log_path)

# Creates a log file to track information of interest
with open(log_path, 'a') as f:
    f.write('Rows dropped with missing GPS points: ')

In [141]:
# List for files excluding ones starting with '.'
filtered_data_list = [file for file in raw_data_list if not file.startswith('.')]

# Removes '.DAT' string from any files that have it
for idx, name in enumerate(filtered_data_list):
    original_name = os.path.join(root_path, 'Original Data', name)
    new_name = os.path.join(root_path, 'Original Data', name.replace('.DAT', ''))
    os.rename(original_name, new_name)
    filtered_data_list[idx] = name.replace('.DAT', '')

In [142]:
total_miles_list = []  # List for total miles per '.csv' file
total_rows_dropped = []  # List for total rows dropped that don't have GPS coordinates
i = 0  # Loop counter for calculating distance (ft)
j = 0  # List counter for '.csv' files

# Goes through all the files in the list
while j < len(filtered_data_list):
    # Create dataframe from a csv file
    df_cis = pd.read_csv(os.path.join(root_path, 'Original Data', filtered_data_list[j]))

    # Drops elevation and PDOP columns
    df_cis = df_cis[['Station', 'On Potential', 'Off Potential',
                     'Comments', 'Latitude', 'Longitude']]

    # Reversing rows for flipped SN
    if reverse:
        # Reversing rows
        df_cis.iloc[:, :] = df_cis.iloc[:, :].values[::-1]

        # Reordering
        df_cis['Station'] = abs(max(df_cis['Station']) - df_cis['Station'])

    # Delete rows that CIS was skipped and reset the index
    df_cis = df_cis.loc[df_cis['On Potential'] != 'SKIP'].reset_index(drop=True)

    # Extract starting stationing number (ft) from '.csv' title
    starting_stationing = float(filtered_data_list[j].split('SN')[1].split()[0].replace('+', ''))

    # Add 'Stationing (ft)' column to hold calculated stationing values
    df_cis['Stationing (ft)'] = ''

    # Convert object columns to numbers
    df_cis['On Potential'] = pd.to_numeric(df_cis['On Potential'], errors='coerce')
    df_cis['Off Potential'] = pd.to_numeric(df_cis['Off Potential'], errors='coerce')

    # Replace empty values with 0
    df_cis['On Potential'].replace(np.nan, 0, inplace=True)
    df_cis['Off Potential'].replace(np.nan, 0, inplace=True)

    # Replace exact values
    df_cis['On Potential'].replace(-0.85, -0.850001, inplace=True)
    df_cis['Off Potential'].replace(-0.85, -0.850001, inplace=True)
    df_cis['Off Potential'].replace(-1.2, -1.20001, inplace=True)

    # Convert positive values to negative
    df_cis['On Potential'] = df_cis['On Potential'].abs() * (-1)
    df_cis['Off Potential'] = df_cis['Off Potential'].abs() * (-1)

    # Divide by 1000 if needed
    if DIVISION_1000:
        df_cis[['On Potential', 'Off Potential']] /= 1000

    # Trim white space from comments
    df_cis['Comments'] = df_cis['Comments'].str.strip()

    # Create 'Distance (ft)' column
    df_cis['Distance (ft)'] = 0.00
    last_index2 = df_cis.last_valid_index()

    # Drop rows that don't have GPS coordinates
    df_cis['Latitude'].replace('', np.nan, inplace=True)
    df_cis.dropna(subset=['Latitude'], inplace=True)
    df_cis.reset_index(drop=True, inplace=True)
    last_index = df_cis.last_valid_index()
    total_rows_dropped.append(last_index2 - last_index)

    # Record number of rows dropped for each file
    with open(log_path, 'a') as f:
        f.write(f"{total_rows_dropped[j]}\n")

    # Infers actual stationing number (xxxx+xx)
    while i <= last_index:
        current_station = df_cis.at[i, 'Station']
        stationing_ft = str(starting_stationing + current_station).split('.')[0]
        string_length = len(stationing_ft)

        if -100 < (starting_stationing + current_station) < 0:
            station_value = stationing_ft.replace('-', '')
            df_cis.at[i, 'Stationing (ft)'] = (
                f" -0{station_value[:string_length - 2]}+{station_value[string_length - 2:]}"
            )
        elif 0 <= (starting_stationing + current_station) < 100:
            df_cis.at[i, 'Stationing (ft)'] = (
                f" 0{stationing_ft[:string_length - 2]}+{stationing_ft[string_length - 2:]}"
            )
        else:
            df_cis.at[i, 'Stationing (ft)'] = (
                f"{stationing_ft[:string_length - 2]}+{stationing_ft[string_length - 2:]}"
            )

        # Last row exception
        if i != last_index:
            # Calculates distance between GPS coordinates and stationing (ft)
            try:
                df_cis.at[i + 1, 'Distance (ft)'] = VincentyInverse(
                    [df_cis.at[i, 'Latitude'], df_cis.at[i, 'Longitude']],
                    [df_cis.at[i + 1, 'Latitude'], df_cis.at[i + 1, 'Longitude']]
                ).ft
            except ZeroDivisionError:
                df_cis.at[i + 1, 'Distance (ft)'] = 0
                
        # Counters
        i += 1

    # Calculates standard deviation between GPS coordinates
    for i in range(1, last_index + 1):
        df_cis.at[i, 'Error'] = (
                df_cis.at[i, 'Distance (ft)'] - df_cis.at[i, 'Station'] +
                df_cis.at[i - 1, 'Station']
        )
        std = df_cis['Error'].std()

    # Drop 'Error' column
    df_cis = df_cis.drop('Error', axis=1)

    # Records total miles in a list
    total_miles_list.append(max(df_cis['Station']) / 5280)

    # Export to csv
    df_cis.to_csv(os.path.join(data_folder_path, filtered_data_list[j]), index=False)

    # Counters
    i = 0
    j += 1

# Create Google Earth Folders

In [143]:
cis_kmz = []
type_folders = []
current_mile = 1
k = 0  # List counter for type folders
j = 0  # List counter for '.csv' files
i = 0  # List counter for '.kmz' files based on miles

In [144]:
# Create CIS survey '.kmz' files per mile #
while j < len(total_miles_list):
    miles_remaining = round(total_miles_list[j], 2)

    # Create type folders
    while miles_remaining > 0:
        # Separate folders by mile
        cis_kmz.append(f"{filtered_data_list[j].split('.CSV', 1)[0]} (Mile {current_mile})")
        cis_kmz[i] = simplekml.Kml()

        # Separate sub folders by type
        type_folders.append(f"{filtered_data_list[j].split('.CSV', 1)[0]} "
                            f"(Mile {current_mile})(On)")
        type_folders.append(f"{filtered_data_list[j].split('.CSV', 1)[0]} "
                            f"(Mile {current_mile})(Off)")
        type_folders.append(f"{filtered_data_list[j].split('.CSV', 1)[0]} "
                            f"(Mile {current_mile})(Comments)")
        type_folders.append(f"{filtered_data_list[j].split('.CSV', 1)[0]} "
                            f"(Mile {current_mile})(-1.2 V)")
        type_folders.append(f"{filtered_data_list[j].split('.CSV', 1)[0]} "
                            f"(Mile {current_mile})(-0.85 V)")
        type_folders[k] = cis_kmz[i].newfolder(name='On')
        type_folders[k + 1] = cis_kmz[i].newfolder(name='Off')
        type_folders[k + 2] = cis_kmz[i].newfolder(name='Comments')
        type_folders[k + 3] = cis_kmz[i].newfolder(name='-1.2 V')
        type_folders[k + 4] = cis_kmz[i].newfolder(name='-0.85 V')

        # Save to file
        kmz_name = f"{filtered_data_list[j].split('.CSV', 1)[0]} (Mile {current_mile}).kmz"
        cis_kmz[i].savekmz(os.path.join(root_path, 'Output', kmz_name))

        # Counters
        current_mile += 1
        miles_remaining -= 1
        if 0 < miles_remaining < 1:
            current_mile = round(total_miles_list[j], 2)
        i += 1
        k += 5

    # Counters
    j += 1
    current_mile = 1

# CIS Data

## On

In [145]:
feet_counter = 5280
style = simplekml.Style()
i = 0  # Loop counter for rows
j = 0  # Loop counter for '.csv' files
k = 0  # Loop counter for type folders (0 = 'On')
kmz_file = 0

# Goes through all '.csv' files
while j < len(filtered_data_list):
    df_cis_on = pd.read_csv(os.path.join(root_path, 'Data', filtered_data_list[j]))
    df_cis_on = df_cis_on[df_cis_on['On Potential'] != 0]
    df_cis_on = df_cis_on[['Station', 'Stationing (ft)', 'Comments', 'Longitude',
                           'Latitude', 'On Potential']].reset_index(drop=True)
    df_cis_on['On Potential'] = df_cis_on['On Potential'] * (-1)
    last_index = df_cis_on.last_valid_index()
    on_measurements = last_index + 1
    miles_remaining = round(total_miles_list[j], 2)

    # Create '.kmz' files for each mile #
    while miles_remaining > 0:
        # Create 3D data points
        while i <= last_index and df_cis_on.at[i, 'Station'] < feet_counter:
            pnt = type_folders[k].newpoint(name=df_cis_on.at[i, 'On Potential'] * (-1),
                                           visibility=DATA_VISIBILITY)
            pnt.style.balloonstyle.text = (
                f"Potential: -{df_cis_on.at[i, 'On Potential']} V\n"
                f"Longitude: {df_cis_on.at[i, 'Longitude']}\n"
                f"Latitude: {df_cis_on.at[i, 'Latitude']}\n"
                f"Station (ft): {df_cis_on.at[i, 'Stationing (ft)']}"
            )
            pnt.coords = [(df_cis_on.at[i, 'Longitude'], df_cis_on.at[i, 'Latitude'],
                           df_cis_on.at[i, 'On Potential'] * SCALE_FACTOR)]
            pnt.altitudemode = simplekml.AltitudeMode.relativetoground
            pnt.style.iconstyle.icon.href = ICON_ON
            pnt.style.iconstyle.scale = ICON_SCALE
            pnt.style.labelstyle.scale = 0
            pnt.extrude = 0

            # Counters
            i += 1

        # Save to file
        kmz_name = f"{filtered_data_list[j].split('.CSV', 1)[0]} (Mile {current_mile}).kmz"
        cis_kmz[kmz_file].savekmz(os.path.join(output_path, kmz_name))

        # Counters
        current_mile += 1
        miles_remaining -= 1
        if 0 < miles_remaining < 1:
            current_mile = round(total_miles_list[j], 2)

        feet_counter += 5280
        k += 5
        kmz_file += 1

    # Counters
    feet_counter = 5280
    current_mile = 1
    i = 0
    j += 1

# Counters
j = 0
kmz_file = 0

In [146]:
# Record duplicate GPS coordinates in a '.csv' file
df_duplicates = df_cis_on[df_cis_on.duplicated(subset=['Longitude', 'Latitude'], keep=False)]
df_duplicates.to_csv(duplicates_path, index=False)

## Off

In [147]:
k = 1  # Loop counter for type folders (1 = 'Off')
off_measurements = 0

# Goes through all '.csv' files
while j < len(filtered_data_list):
    df_cis_off = pd.read_csv(os.path.join(root_path, 'Data', filtered_data_list[j]))
    df_cis_off = df_cis_off[df_cis_off['Off Potential'] != 0]
    df_cis_off = df_cis_off[['Station', 'Stationing (ft)', 'Longitude', 'Latitude',
                             'Off Potential']].reset_index(drop=True)
    df_cis_off['Off Potential'] = df_cis_off['Off Potential'] * (-1)
    last_index = df_cis_off.last_valid_index()
    off_measurements = last_index + 1
    miles_remaining = round(total_miles_list[j], 2)

    # Create '.kmz' files for each mile #
    while miles_remaining > 0:
        # Create 3D data points
        while i <= last_index and df_cis_off.at[i, 'Station'] < feet_counter:
            # Break loop if there are no 'Off' potentials
            if df_cis_off.shape[0] == 0:
                break
            pnt = type_folders[k].newpoint(name=df_cis_off.at[i, 'Off Potential'] * (-1),
                                           visibility=DATA_VISIBILITY)
            pnt.style.balloonstyle.text = (
                f"Potential: -{df_cis_off.at[i, 'Off Potential']} V\n"
                f"Longitude: {df_cis_off.at[i, 'Longitude']}\n"
                f"Latitude: {df_cis_off.at[i, 'Latitude']}\n"
                f"Station (ft): {df_cis_off.at[i, 'Stationing (ft)']}"
            )
            pnt.coords = [(df_cis_off.at[i, 'Longitude'], df_cis_off.at[i, 'Latitude'],
                           df_cis_off.at[i, 'Off Potential'] * SCALE_FACTOR)]
            pnt.altitudemode = simplekml.AltitudeMode.relativetoground
            pnt.style.iconstyle.icon.href = ICON_OFF
            pnt.style.iconstyle.scale = ICON_SCALE
            pnt.style.linestyle.width = 0.01
            pnt.style.linestyle.color = simplekml.Color.rgb(255, 255, 255, round(255 * 0.15))
            pnt.style.labelstyle.scale = 0
            pnt.extrude = 1

            # Counters
            i += 1

        # Save to file
        kmz_name = f"{filtered_data_list[j].split('.CSV', 1)[0]} (Mile {current_mile}).kmz"
        cis_kmz[kmz_file].savekmz(os.path.join(output_path, kmz_name))

        # Counters
        current_mile += 1
        miles_remaining -= 1
        if 0 < miles_remaining < 1:
            current_mile = round(total_miles_list[j], 2)

        feet_counter += 5280
        k += 5
        kmz_file += 1

    # Counters
    feet_counter = 5280
    current_mile = 1
    i = 0
    j += 1

# Counters
j = 0
kmz_file = 0

## Comments

In [148]:
k = 2  # Loop counter for type folders (2 = 'Comments')

# Goes through all '.csv' files
while j < len(filtered_data_list):
    df_cis_comments = pd.read_csv(os.path.join(root_path, 'Data', filtered_data_list[j]))
    df_cis_comments.dropna(inplace=True)
    df_cis_comments.reset_index(drop=True, inplace=True)
    last_index = df_cis_comments.last_valid_index()
    miles_remaining = round(total_miles_list[j], 2)

    # Create '.kmz' files for each mile #
    while miles_remaining > 0:
        # Create 3D data points
        while i <= last_index and df_cis_comments.at[i, 'Station'] < \
                feet_counter:
            # Break loop if there are no comments
            if df_cis_comments.shape[0] == 0:
                break
            pnt = type_folders[k].newpoint(name=df_cis_comments.at[i, 'Comments'],
                                           visibility=DATA_VISIBILITY)
            if df_cis_comments.at[i, 'On Potential'] == 0:
                potential = df_cis_comments.at[i, 'Off Potential'].__str__() + ' V (Off)'
            else:
                potential = df_cis_comments.at[i, 'On Potential'].__str__() + ' V (On)'
            pnt.style.balloonstyle.text = (
                f"Potential: {potential}\n"
                f"Longitude: {df_cis_comments.at[i, 'Longitude']}\n"
                f"Latitude: {df_cis_comments.at[i, 'Latitude']}\n"
                f"Station (ft): {df_cis_comments.at[i, 'Stationing (ft)']}\n"
                f"Comment: {df_cis_comments.at[i, 'Comments']}"
            )
            pnt.coords = [(df_cis_comments.at[i, 'Longitude'],
                           df_cis_comments.at[i, 'Latitude'], 0)]
            pnt.altitudemode = simplekml.AltitudeMode.relativetoground
            pnt.style.iconstyle.icon.href = ICON_COMMENTS
            pnt.style.iconstyle.scale = ICON_SCALE * 1.5
            pnt.style.labelstyle.scale = 0.5

            # Counters
            i += 1

        # Save to file
        kmz_name = f"{filtered_data_list[j].split('.CSV', 1)[0]} (Mile {current_mile}).kmz"
        cis_kmz[kmz_file].savekmz(os.path.join(output_path, kmz_name))

        # Counters
        current_mile += 1
        miles_remaining -= 1
        if 0 < miles_remaining < 1:
            current_mile = round(total_miles_list[j], 2)

        feet_counter += 5280
        k += 5
        kmz_file += 1

    # Counters
    feet_counter = 5280
    current_mile = 1
    i = 0
    j += 1

# Counters
j = 0
kmz_file = 0

## -1.2 V

In [149]:
k = 3  # Loop counter for type folders (3 = '-1.2 V')

# Goes through all '.csv' files
while j < len(filtered_data_list):
    df_cis_1200 = pd.read_csv(os.path.join(root_path, 'Data', filtered_data_list[j]))
    last_index = df_cis_1200.last_valid_index()
    miles_remaining = round(total_miles_list[j], 2)

    # Create '.kmz' files for each mile #
    while miles_remaining > 0:
        # Create 3D data points
        while i <= last_index and df_cis_1200.at[i, 'Station'] < feet_counter:
            pnt = type_folders[k].newpoint(name='-1.200', visibility=DATA_VISIBILITY)
            pnt.style.balloonstyle.text = 'Potential: -1.2 V'
            pnt.coords = [(df_cis_1200.at[i, 'Longitude'], df_cis_1200.at[i, 'Latitude'],
                           1.2 * SCALE_FACTOR)]
            pnt.altitudemode = simplekml.AltitudeMode.relativetoground
            pnt.style.iconstyle.icon.href = ICON_1200
            pnt.style.iconstyle.scale = ICON_SCALE
            pnt.style.labelstyle.scale = 0
            pnt.extrude = 0

            # Counters
            i += 1

        # Save to file
        kmz_name = f"{filtered_data_list[j].split('.CSV', 1)[0]} (Mile {current_mile}).kmz"
        cis_kmz[kmz_file].savekmz(os.path.join(output_path, kmz_name))

        # Counters
        current_mile += 1
        miles_remaining -= 1
        if 0 < miles_remaining < 1:
            current_mile = round(total_miles_list[j], 2)

        feet_counter += 5280
        k += 5
        kmz_file += 1

    # Counters
    feet_counter = 5280
    current_mile = 1
    i = 0
    j += 1

# Counters
j = 0
kmz_file = 0

## -0.85 V

In [150]:
k = 4  # Loop counter for type folders (4 = '-0.85 V')

# Goes through all '.csv' files
while j < len(filtered_data_list):
    df_cis_850 = pd.read_csv(os.path.join(root_path, 'Data', filtered_data_list[j]))
    last_index = df_cis_850.last_valid_index()
    miles_remaining = round(total_miles_list[j], 2)

    # Create '.kmz' files for each mile #
    while miles_remaining > 0:
        # Create 3D data points
        while i <= last_index and df_cis_850.at[i, 'Station'] < feet_counter:
            pnt = type_folders[k].newpoint(name='-0.850', visibility=DATA_VISIBILITY)
            pnt.style.balloonstyle.text = 'Potential: -0.85 V'
            pnt.coords = [(df_cis_850.at[i, 'Longitude'], df_cis_850.at[i, 'Latitude'],
                           0.85 * SCALE_FACTOR)]
            pnt.altitudemode = simplekml.AltitudeMode.relativetoground
            pnt.style.iconstyle.icon.href = ICON_850
            pnt.style.iconstyle.scale = ICON_SCALE
            pnt.style.labelstyle.scale = 0
            pnt.extrude = 0
            pnt.style.linestyle.width = 0.01
            pnt.style.linestyle.color = simplekml.Color.rgb(255, 255, 255, round(255 * 0.15))

            # Counters
            i += 1

        # Save to file
        kmz_name = f"{filtered_data_list[j].split('.CSV', 1)[0]} (Mile {current_mile}).kmz"
        cis_kmz[kmz_file].savekmz(os.path.join(output_path, kmz_name))

        # Counters
        current_mile += 1
        miles_remaining -= 1
        if 0 < miles_remaining < 1:
            current_mile = round(total_miles_list[j], 2)

        feet_counter += 5280
        k += 5
        kmz_file += 1

    # Counters
    feet_counter = 5280 
    current_mile = 1
    i = 0
    j += 1

# Counters
j = 0
kmz_file = 0

## Assign SN to KMZs

In [151]:
# TODO: When renaming kmzs ensure following kmz has one SN above previous

In [152]:
# Sort kmz files in folder within a list
kmz_files = glob.glob(os.path.join(output_path, '*.kmz'))
kmz_files = [os.path.basename(file) for file in kmz_files]
kmz_files = natsorted(kmz_files)

In [153]:
sn_list = [''] * (math.ceil(total_miles_list[0]) + 1)
station_list = np.arange(0, math.ceil(total_miles_list[0]) + 1)
last_index = len(station_list)
station_list[0] = starting_stationing

In [154]:
# Assign a value of mile (ft) per element
for i in range(1, last_index):
    station_list[i] = station_list[i - 1] + 5280
    
# Last row
station_list[last_index - 1] = ((total_miles_list[0] - last_index + 2) *
                                5280 + station_list[last_index - 2])

if starting_stationing != 0:
    # Find closest station number
    for i in range(0, last_index):
        closest_index = (df_cis['Station'] + starting_stationing - station_list[i]).abs().argmin()
        sn_list[i] = f"SN{df_cis['Stationing (ft)'].loc[closest_index]}".replace(' ', '')

# Name station number ranges
for i in range(0, last_index - 1):
    sn_list[i] = (f"{filtered_data_list[0].split('SN')[0][:-1]} "
                  f"({sn_list[i]} TO {sn_list[i + 1]}).kmz")

# Drop last row
sn_list = sn_list[:-1]

# Rename '.kmz' files
for i in range(0, last_index - 1):
    os.rename(os.path.join(output_path, kmz_files[i]), os.path.join(output_path, sn_list[i]))

# Areas of Interest

## Less Negative than -0.85V "On"

In [155]:
# Combines all '.csv' files in folder
df_cis_filtered = pd.concat(
    (pd.read_csv(f) for f in glob.glob(os.path.join(data_folder_path, r'*.CSV'))),
    ignore_index=True
)
df_cis_filtered = df_cis_filtered[df_cis_filtered['On Potential'] != 0]
df_cis_filtered = df_cis_filtered[['Station', 'Stationing (ft)', 'Longitude', 'Latitude',
                                   'On Potential']].reset_index(drop=True)
df_cis_filtered['Crossing Point'] = ''
last_index = df_cis_filtered.last_valid_index()

In [156]:
i = 1  # Loop counter for rows

if df_cis_filtered.shape[0] != 0:
    # Finds crossing points
    while i < last_index:
        # First data point
        if i == 1 and df_cis_filtered.at[0, 'On Potential'] >= -0.85:
            df_cis_filtered.at[0, 'Crossing Point'] = 'X'

        # Second data point
        elif i == 2 and df_cis_filtered.at[1, 'On Potential'] >= -0.85:
            df_cis_filtered.at[0, 'Crossing Point'] = 'X'

        # Last data point
        elif (i == last_index - 1 and
              df_cis_filtered.at[last_index, 'On Potential'] >= -0.85):
            df_cis_filtered.at[last_index, 'Crossing Point'] = 'X'

        # _'_
        elif ((df_cis_filtered.at[i - 1, 'On Potential'] > -0.85) and
              (df_cis_filtered.at[i, 'On Potential'] <= -0.85) and
              (df_cis_filtered.at[i + 1, 'On Potential'] > -0.85) and
              i >= 1):
            df_cis_filtered.at[i, 'Crossing Point'] = 'XX'

        # '_
        elif ((df_cis_filtered.at[i + 1, 'On Potential'] > -0.85) and
              (df_cis_filtered.at[i, 'On Potential'] <= -0.85)):
            df_cis_filtered.at[i, 'Crossing Point'] = 'X'

        # _'
        elif ((df_cis_filtered.at[i + 1, 'On Potential'] < -0.85) and
              (df_cis_filtered.at[i, 'On Potential'] >= -0.85)):
            df_cis_filtered.at[i + 1, 'Crossing Point'] = 'X'

        # Counters
        i += 1

    # Drops rows that are empty
    df_cis_filtered = (df_cis_filtered[df_cis_filtered['Crossing Point'] != '']
                       .reset_index(drop=True))

In [157]:
if df_cis_filtered.shape[0] != 0:
    # Replicates rows that have 'XX'
    df_cis_filtered = df_cis_filtered.loc[df_cis_filtered.index
    .repeat(df_cis_filtered['Crossing Point'].isin(['XX']).add(1))].reset_index(drop=True)
    last_index = int((df_cis_filtered.last_valid_index() + 1) / 2)

    # Create report dataframe
    df_cis_report = pd.DataFrame(
        index=np.arange(last_index),
        columns=['Station', 'Station Number', 'Latitude', 'Longitude',
                 'Station', 'Station Number', 'Latitude', 'Longitude']
    )
    df_cis_report['Length (ft)'] = 0.0
    df_cis_report['Comments'] = ''

    # Counters
    i = 0
    j = 0

    # Structure data
    while j < int((df_cis_filtered.last_valid_index() + 1) / 2):
        # Start
        df_cis_report.iat[j, 0] = df_cis_filtered.at[i, 'Station']
        df_cis_report.iat[j, 1] = df_cis_filtered.at[i, 'Stationing (ft)']
        df_cis_report.iat[j, 2] = df_cis_filtered.at[i, 'Latitude']
        df_cis_report.iat[j, 3] = df_cis_filtered.at[i, 'Longitude']

        # End
        df_cis_report.iat[j, 4] = df_cis_filtered.at[i + 1, 'Station']
        df_cis_report.iat[j, 5] = df_cis_filtered.at[i + 1, 'Stationing (ft)']
        df_cis_report.iat[j, 6] = df_cis_filtered.at[i + 1, 'Latitude']
        df_cis_report.iat[j, 7] = df_cis_filtered.at[i + 1, 'Longitude']

        # Length
        df_cis_report.iat[j, 8] = df_cis_report.iat[j, 4] - df_cis_report.iat[j, 0]

        # Counters
        j += 1
        i += 2

    # Deletes station columns
    df_cis_report = df_cis_report.drop(df_cis_report.iloc[:, [0, 4]], axis=1)

    # Save to file
    excel_name = f"CIS Exception Report ({filtered_data_list[0].split('.CSV')[0]}).xlsx"

    with pd.ExcelWriter(os.path.join(output_path, excel_name),
                        mode='w', engine='openpyxl') as writer:
        df_cis_report.to_excel(writer, startrow=4,
                               sheet_name='Less Negative than -0.85V (On)',
                               index=False)

In [158]:
if df_cis_filtered.shape[0] != 0:
    wb = load_workbook(os.path.join(output_path, excel_name))
    sheet = wb['Less Negative than -0.85V (On)']
    # Company name with pipeline ID
    sheet.cell(1, 1).value = f"{COMPANY_NAME} ({filtered_data_list[0].split('SN')[0][:-1]})"

    # Station numbers surveyed
    sheet.cell(2, 1).value = (
        f"{filtered_data_list[0].split('SN')[1].split()[0]} to "
        f"{filtered_data_list[0].split('SN')[2].split('.')[0]}"
    )

    # Start
    sheet.cell(4, 1).value = 'Start'

    # End
    sheet.cell(4, 4).value = 'End'

    # Total pipeline distance with issues
    total_feet = round(sum(total_miles_list * 5280))
    total_miles = round(sum(total_miles_list), 2)
    sheet.cell(last_index + 7, 1).value = (
        f"Total pipeline distance surveyed = {total_feet} feet or {total_miles} miles"
    )
    total_length = df_cis_report['Length (ft)'].sum()
    length_percent = round(total_length / sum(total_miles_list * 5280) * 100, 2)
    sheet.cell(last_index + 8, 1).value = (
        f"Total pipeline distance less negative than -0.85V 'On' = "
        f"{total_length} feet ({length_percent} %)"
    )

In [159]:
if df_cis_filtered.shape[0] != 0:
    # Font
    sheet.cell(1, 1).font = Font(size=14, bold=True)
    sheet.cell(2, 1).font = Font(italic=True)
    sheet.cell(4, 1).font = Font(bold=True)
    sheet.cell(4, 4).font = Font(bold=True)
    sheet.cell(4, 7).font = Font(bold=True)
    sheet.cell(4, 8).font = Font(bold=True)
    sheet.cell(last_index + 7, 1).font = Font(bold=True)
    sheet.cell(last_index + 8, 1).font = Font(bold=True)

    # Merging
    sheet.merge_cells('A1:H1')
    sheet.merge_cells('A2:H2')
    sheet.merge_cells('A4:C4')
    sheet.merge_cells('D4:F4')
    sheet.cell(4, 7).value = sheet.cell(5, 7).value
    sheet.merge_cells('G4:G5')
    sheet.cell(4, 8).value = sheet.cell(5, 8).value
    sheet.merge_cells('H4:H5')

    # Alignment
    i = 0

    while i < (last_index + 5):
        for c in sheet['A1:H' + str(last_index + 5)][i]:
            c.alignment = Alignment(horizontal='center', vertical='center')

        # Counters
        i += 1

    # Borders
    i = 0
    thin = Side(border_style='thin', color='000000')
    white_border = Side(border_style='thin', color='FFFFFF')

    while i < (last_index + 5):
        # 3rd row
        if i == 2:
            for c in sheet['A1:H' + str(last_index + 5)][i]:
                c.border = Border(left=white_border, right=white_border)

            # Counters
            i += 1

        for c in sheet['A1:H' + str(last_index + 5)][i]:
            c.border = Border(top=thin, left=thin, right=thin, bottom=thin)

        # Counters
        i += 1

    # Column widths
    for col in 'ABCDEFG':
        sheet.column_dimensions[col].width = 15

    sheet.column_dimensions['H'].width = 50

In [160]:
if df_cis_filtered.shape[0] != 0:
    wb.save(os.path.join(output_path, excel_name))

## Less Negative than -0.85V "Off"

In [161]:
# Combines all '.csv' files at specified folder
df_cis_filtered = pd.concat(
    (pd.read_csv(f) for f in glob.glob(os.path.join(data_folder_path, r'*.CSV'))),
    ignore_index=True
)
df_cis_filtered = df_cis_filtered[df_cis_filtered['Off Potential'] != 0]
df_cis_filtered = df_cis_filtered[['Station', 'Stationing (ft)', 'Longitude',
                                   'Latitude', 'Off Potential']].reset_index(drop=True)
df_cis_filtered['Crossing Point'] = ''
last_index = df_cis_filtered.last_valid_index()

In [162]:
i = 1  # Loop counter for rows

if df_cis_filtered.shape[0] != 0:
    # Finds crossing points
    while i < last_index:
        # First data point
        if i == 1 and df_cis_filtered.at[0, 'Off Potential'] >= -0.85:
            df_cis_filtered.at[0, 'Crossing Point'] = 'X'

        # Second data point
        elif i == 2 and df_cis_filtered.at[1, 'Off Potential'] >= -0.85:
            df_cis_filtered.at[0, 'Crossing Point'] = 'X'

        # Last data point
        elif (i == last_index - 1 and
              df_cis_filtered.at[last_index, 'Off Potential'] >= -0.85):
            df_cis_filtered.at[last_index, 'Crossing Point'] = 'X'

        # _'_
        elif ((df_cis_filtered.at[i - 1, 'Off Potential'] > -0.85) and
              (df_cis_filtered.at[i, 'Off Potential'] <= -0.85) and
              (df_cis_filtered.at[i + 1, 'Off Potential'] > -0.85) and
              i >= 1):
            df_cis_filtered.at[i, 'Crossing Point'] = 'XX'

        # '_
        elif ((df_cis_filtered.at[i + 1, 'Off Potential'] > -0.85) and
              (df_cis_filtered.at[i, 'Off Potential'] <= -0.85)):
            df_cis_filtered.at[i, 'Crossing Point'] = 'X'

        # _'
        elif ((df_cis_filtered.at[i + 1, 'Off Potential'] < -0.85) and
              (df_cis_filtered.at[i, 'Off Potential'] >= -0.85)):
            df_cis_filtered.at[i + 1, 'Crossing Point'] = 'X'

        # Counters
        i += 1

    df_cis_filtered = (df_cis_filtered[df_cis_filtered['Crossing Point'] != '']
                       .reset_index(drop=True))

In [163]:
if df_cis_filtered.shape[0] != 0:
    # Replicates rows that have 'XX'
    df_cis_filtered = df_cis_filtered.loc[df_cis_filtered.index
    .repeat(df_cis_filtered['Crossing Point'].isin(['XX']).add(1))].reset_index(drop=True)
    last_index = int((df_cis_filtered.last_valid_index() + 1) / 2)

    # Create report dataframe
    df_cis_report = pd.DataFrame(
        index=np.arange(last_index),
        columns=['Station', 'Station Number', 'Latitude', 'Longitude',
                 'Station', 'Station Number', 'Latitude', 'Longitude']
    )
    df_cis_report['Length (ft)'] = 0.0
    df_cis_report['Comments'] = ''

    # Counters
    i = 0
    j = 0

    # Structure data
    while j < int((df_cis_filtered.last_valid_index() + 1) / 2):
        # Start
        df_cis_report.iat[j, 0] = df_cis_filtered.at[i, 'Station']
        df_cis_report.iat[j, 1] = df_cis_filtered.at[i, 'Stationing (ft)']
        df_cis_report.iat[j, 2] = df_cis_filtered.at[i, 'Latitude']
        df_cis_report.iat[j, 3] = df_cis_filtered.at[i, 'Longitude']

        # End
        df_cis_report.iat[j, 4] = df_cis_filtered.at[i + 1, 'Station']
        df_cis_report.iat[j, 5] = df_cis_filtered.at[i + 1, 'Stationing (ft)']
        df_cis_report.iat[j, 6] = df_cis_filtered.at[i + 1, 'Latitude']
        df_cis_report.iat[j, 7] = df_cis_filtered.at[i + 1, 'Longitude']

        # Length
        df_cis_report.iat[j, 8] = df_cis_report.iat[j, 4] - df_cis_report.iat[j, 0]

        # Counters
        j += 1
        i += 2

    # Deletes station columns
    df_cis_report = df_cis_report.drop(df_cis_report.iloc[:, [0, 4]], axis=1)

    # Save to file
    # Save to file
    excel_name = f"CIS Exception Report ({filtered_data_list[0].split('.CSV')[0]}).xlsx"
    writing_mode = 'w'

    if os.path.exists(os.path.join(output_path, excel_name)):
        writing_mode = 'a'

    with pd.ExcelWriter(os.path.join(output_path, excel_name),
                        mode=writing_mode, engine='openpyxl') as writer:
        df_cis_report.to_excel(writer, startrow=4,
                               sheet_name='Less Negative than -0.85V (Off)',
                               index=False)

In [164]:
if df_cis_filtered.shape[0] != 0:
    wb = load_workbook(os.path.join(output_path, excel_name))
    sheet = wb['Less Negative than -0.85V (Off)']

    # Company name with pipeline ID
    sheet.cell(1, 1).value = f"{COMPANY_NAME} ({filtered_data_list[0].split('SN')[0][:-1]})"

    # Station numbers surveyed
    sheet.cell(2, 1).value = (
        f"{filtered_data_list[0].split('SN')[1].split()[0]} to "
        f"{filtered_data_list[0].split('SN')[2].split('.')[0]}"
    )

    # Start
    sheet.cell(4, 1).value = 'Start'

    # End
    sheet.cell(4, 4).value = 'End'

    # Total pipeline distance with issues
    total_feet = round(sum(total_miles_list * 5280))
    total_miles = round(sum(total_miles_list), 2)
    sheet.cell(last_index + 7, 1).value = (
        f"Total pipeline distance surveyed = {total_feet} feet or {total_miles} miles"
    )
    total_length = df_cis_report['Length (ft)'].sum()
    length_percent = round(total_length / sum(total_miles_list * 5280) * 100, 2)
    sheet.cell(last_index + 8, 1).value = (
        f"Total pipeline distance less negative than -0.85V 'Off' = "
        f"{total_length} feet ({length_percent} %)"
    )

In [165]:
if df_cis_filtered.shape[0] != 0:
    # Font
    sheet.cell(1, 1).font = Font(size=14, bold=True)
    sheet.cell(2, 1).font = Font(italic=True)
    sheet.cell(4, 1).font = Font(bold=True)
    sheet.cell(4, 4).font = Font(bold=True)
    sheet.cell(4, 7).font = Font(bold=True)
    sheet.cell(4, 8).font = Font(bold=True)
    sheet.cell(last_index + 7, 1).font = Font(bold=True)
    sheet.cell(last_index + 8, 1).font = Font(bold=True)

    # Merging
    sheet.merge_cells('A1:H1')
    sheet.merge_cells('A2:H2')
    sheet.merge_cells('A4:C4')
    sheet.merge_cells('D4:F4')
    sheet.cell(4, 7).value = sheet.cell(5, 7).value
    sheet.merge_cells('G4:G5')
    sheet.cell(4, 8).value = sheet.cell(5, 8).value
    sheet.merge_cells('H4:H5')

    # Alignment
    i = 0

    while i < (last_index + 5):
        for c in sheet['A1:H' + str(last_index + 5)][i]:
            c.alignment = Alignment(horizontal='center', vertical='center')

        # Counters
        i += 1

    # Borders
    i = 0
    thin = Side(border_style='thin', color='000000')
    white_border = Side(border_style='thin', color='FFFFFF')

    while i < (last_index + 5):
        # 3rd row
        if i == 2:
            for c in sheet['A1:H' + str(last_index + 5)][i]:
                c.border = Border(left=white_border, right=white_border)

            # Counters
            i += 1

        for c in sheet['A1:H' + str(last_index + 5)][i]:
            c.border = Border(top=thin, left=thin, right=thin, bottom=thin)

        # Counters
        i += 1

    # Column widths
    for col in 'ABCDEFG':
        sheet.column_dimensions[col].width = 15

    sheet.column_dimensions['H'].width = 50

In [166]:
if df_cis_filtered.shape[0] != 0:
    wb.save(os.path.join(output_path, excel_name))

## More Negative than -1.2V "Off"

In [167]:
# Combines all '.csv' files at specified folder
df_cis_filtered = pd.concat(
    (pd.read_csv(f) for f in glob.glob(os.path.join(data_folder_path, r'*.CSV'))),
    ignore_index=True
)
df_cis_filtered = df_cis_filtered[df_cis_filtered['Off Potential'] != 0]
df_cis_filtered = df_cis_filtered[['Station', 'Stationing (ft)', 'Longitude', 'Latitude',
                                   'Off Potential']].reset_index(drop=True)
df_cis_filtered['Crossing Point'] = ''
last_index = df_cis_filtered.last_valid_index()

In [168]:
i = 1  # Loop counter for rows

# Finds crossing points
if df_cis_filtered.shape[0] != 0:
    while i < last_index:
        # First data point
        if i == 1 and df_cis_filtered.at[0, 'Off Potential'] <= -1.2:
            df_cis_filtered.at[0, 'Crossing Point'] = 'X'

        # Second data point
        elif i == 2 and df_cis_filtered.at[1, 'Off Potential'] <= -1.2:
            df_cis_filtered.at[0, 'Crossing Point'] = 'X'

        # Last data point
        elif (i == last_index - 1 and
              df_cis_filtered.at[last_index, 'Off Potential'] <= -1.2):
            df_cis_filtered.at[last_index, 'Crossing Point'] = 'X'

        # '_'
        elif ((df_cis_filtered.at[i - 1, 'Off Potential'] < -1.2) and
              (df_cis_filtered.at[i, 'Off Potential'] >= -1.2) and
              (df_cis_filtered.at[i + 1, 'Off Potential'] < -1.2) and
              i >= 1):
            df_cis_filtered.at[i, 'Crossing Point'] = 'XX'

        # '_
        elif ((df_cis_filtered.at[i + 1, 'Off Potential'] > -1.2) and
              (df_cis_filtered.at[i, 'Off Potential'] <= -1.2)):
            df_cis_filtered.at[i + 1, 'Crossing Point'] = 'X'

        # _'
        elif ((df_cis_filtered.at[i + 1, 'Off Potential'] < -1.2) and
              (df_cis_filtered.at[i, 'Off Potential'] >= -1.2)):
            df_cis_filtered.at[i, 'Crossing Point'] = 'X'

        # Counters
        i += 1

    df_cis_filtered = (df_cis_filtered[df_cis_filtered['Crossing Point'] != '']
                       .reset_index(drop=True))

In [169]:
if df_cis_filtered.shape[0] != 0:
    # Replicates rows that have 'XX'
    df_cis_filtered = df_cis_filtered.loc[df_cis_filtered.index
    .repeat(df_cis_filtered['Crossing Point'].isin(['XX']).add(1))].reset_index(drop=True)
    last_index = int((df_cis_filtered.last_valid_index() + 1) / 2)

    # Create report dataframe
    df_cis_report = pd.DataFrame(
        index=np.arange(last_index),
        columns=['Station', 'Station Number', 'Latitude', 'Longitude',
                 'Station', 'Station Number', 'Latitude', 'Longitude']
    )
    df_cis_report['Length (ft)'] = 0.0
    df_cis_report['Comments'] = ''

    # Counters
    i = 0
    j = 0

    # Structure data
    while j < int((df_cis_filtered.last_valid_index() + 1) / 2):
        # Start
        df_cis_report.iat[j, 0] = df_cis_filtered.at[i, 'Station']
        df_cis_report.iat[j, 1] = df_cis_filtered.at[i, 'Stationing (ft)']
        df_cis_report.iat[j, 2] = df_cis_filtered.at[i, 'Latitude']
        df_cis_report.iat[j, 3] = df_cis_filtered.at[i, 'Longitude']

        # End
        df_cis_report.iat[j, 4] = df_cis_filtered.at[i + 1, 'Station']
        df_cis_report.iat[j, 5] = df_cis_filtered.at[i + 1, 'Stationing (ft)']
        df_cis_report.iat[j, 6] = df_cis_filtered.at[i + 1, 'Latitude']
        df_cis_report.iat[j, 7] = df_cis_filtered.at[i + 1, 'Longitude']

        # Length
        df_cis_report.iat[j, 8] = df_cis_report.iat[j, 4] - df_cis_report.iat[j, 0]

        # Counters
        j += 1
        i += 2

    # Deletes station columns
    df_cis_report = df_cis_report.drop(df_cis_report.iloc[:, [0, 4]], axis=1)

    # Save to file
    excel_name = f"CIS Exception Report ({filtered_data_list[0].split('.CSV')[0]}).xlsx"
    writing_mode = 'w'

    if os.path.exists(os.path.join(output_path, excel_name)):
        writing_mode = 'a'

    with pd.ExcelWriter(os.path.join(output_path, excel_name),
                        mode=writing_mode, engine='openpyxl') as writer:
        df_cis_report.to_excel(writer, startrow=4,
                               sheet_name='More Negative than -1.2V (Off)',
                               index=False)

In [170]:
if df_cis_filtered.shape[0] != 0:
    wb = load_workbook(os.path.join(output_path, excel_name))
    sheet = wb['More Negative than -1.2V (Off)']

    # Company name with pipeline ID
    sheet.cell(1, 1).value = f"{COMPANY_NAME} ({filtered_data_list[0].split('SN')[0][:-1]})"

    # Station numbers surveyed
    sheet.cell(2, 1).value = (
        f"{filtered_data_list[0].split('SN')[1].split()[0]} to "
        f"{filtered_data_list[0].split('SN')[2].split('.')[0]}"
    )

    # Start
    sheet.cell(4, 1).value = 'Start'

    # End
    sheet.cell(4, 4).value = 'End'

    # Total pipeline distance with issues
    total_feet = round(sum(total_miles_list * 5280))
    total_miles = round(sum(total_miles_list), 2)
    sheet.cell(last_index + 7, 1).value = (
        f"Total pipeline distance surveyed = {total_feet} feet or {total_miles} miles"
    )
    total_length = df_cis_report['Length (ft)'].sum()
    length_percent = round(total_length / sum(total_miles_list * 5280) * 100, 2)
    sheet.cell(last_index + 8, 1).value = (
        f"Total pipeline distance more negative than -1.2V 'Off' = "
        f"{total_length} feet ({length_percent} %)"
    )

In [171]:
if df_cis_filtered.shape[0] != 0:
    # Font
    sheet.cell(1, 1).font = Font(size=14, bold=True)
    sheet.cell(2, 1).font = Font(italic=True)
    sheet.cell(4, 1).font = Font(bold=True)
    sheet.cell(4, 4).font = Font(bold=True)
    sheet.cell(4, 7).font = Font(bold=True)
    sheet.cell(4, 8).font = Font(bold=True)
    sheet.cell(last_index + 7, 1).font = Font(bold=True)
    sheet.cell(last_index + 8, 1).font = Font(bold=True)

    # Merging
    sheet.merge_cells('A1:H1')
    sheet.merge_cells('A2:H2')
    sheet.merge_cells('A4:C4')
    sheet.merge_cells('D4:F4')
    sheet.cell(4, 7).value = sheet.cell(5, 7).value
    sheet.merge_cells('G4:G5')
    sheet.cell(4, 8).value = sheet.cell(5, 8).value
    sheet.merge_cells('H4:H5')

    # Alignment
    i = 0

    while i < (last_index + 5):
        for c in sheet['A1:H' + str(last_index + 5)][i]:
            c.alignment = Alignment(horizontal='center', vertical='center')

        # Counters
        i += 1

    # Borders
    i = 0
    thin = Side(border_style='thin', color='000000')
    white_border = Side(border_style='thin', color='FFFFFF')

    while i < (last_index + 5):
        # 3rd row
        if i == 2:
            for c in sheet['A1:H' + str(last_index + 5)][i]:
                c.border = Border(left=white_border, right=white_border)

            # Counters
            i += 1

        for c in sheet['A1:H' + str(last_index + 5)][i]:
            c.border = Border(top=thin, left=thin, right=thin, bottom=thin)

        # Counters
        i += 1

    # Column widths
    for col in 'ABCDEFG':
        sheet.column_dimensions[col].width = 15

    sheet.column_dimensions['H'].width = 50

In [172]:
if df_cis_filtered.shape[0] != 0:
    wb.save(os.path.join(output_path, excel_name))

In [173]:
# Current date and time
e = datetime.datetime.now()

# Modify log file with information of interest
with open(log_path, 'a') as f:
    f.write(f"Duplicated GPS Pairs: {df_duplicates['Station'].value_counts().sum()}\n")
    f.write(f"Standard Deviation (GPS): {round(std, 3)}\n")
    f.write(f"On Measurements: {on_measurements}\n")
    f.write(f"Off Measurements: {off_measurements}\n")
    f.write(f"Ratio: {on_measurements / off_measurements}\n\n")
    f.write(f"User: {socket.gethostname()}\n")
    f.write('%s/%s/%s, %s:%s:%s' % (e.month, e.day, e.year, e.hour, e.minute, e.second))
    f.write('\n')
    f.write(f"Execution Time = {round(time.time() - start_time)} seconds")